# Experiment

Objective: Given the space around the controlled aircraft, predict the next action: categorical encoding of (change in speed, change in altitude, *final heading*) between **current time** and the **next known time of aircraft**.

Preprocessing: For each unique timestamp, pad last known location of aircraft.

Note: Model takes only the space around aircraft into account. It does not consider the absolute ground speed, altitude and heading.

In [ ]:
# This code is for supervised learning using previous state to predict current action
# Code from gym-flight to be reused as much as possible
import tensorflow as tf
import sys
import os
sys.path.append('../gym-flight/')
import gym_flight
from gym_flight.utils.numpy_util import Sparse3dArray, np_3darray_to_Sparse3dArray
import numpy as np
import math
import pandas as pd
import multiprocessing as mp
from full3d_util import get_range_df, bind, unlist
from multiclass import *
from full3d_util import get_action_discrete, get_action_continuous, fix_XY, train_test_split
from aircraft_center_3d_util import get_space_around_ac, get_env_action_aircraft, get_X_Y, pad_zeros
import pickle

splits = mp.cpu_count() - 1

In [ ]:
import logging
logging.basicConfig(
    filename='log.txt',
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
half_x_length = 10
half_y_length = 14
half_z_length = 24
state_dim = [half_x_length, half_y_length, half_z_length]
x_length = 100
y_length = 100
z_length = 100
ts_range = 2
time_dim = 1
state_dim = [2 * half_x_length + 1, 2 * half_y_length + 1, 2 * half_z_length + 1]
action_size = 168
learning_rate = 0.025
flight_data_path = "../gym-flight/data/processed_jfk.csv"
dtype_dict = {"id": str, "ts": np.int16, "lat": np.float32, "lon": np.float32, "altitude": np.float32, "speed": np.float32, "x": np.int16, "y": np.int16, "z": np.int16, "is_landing": np.int8}
flight_data = pd.read_csv(flight_data_path, dtype = dtype_dict)

## Checking the amount of distance / climb of the aircraft in 1 min (unit time step)

Doubling the value of grid because an aircraft may have data from the beginning of the minute whereas the other aircraft may have data from the end of the next minute - time gap is almost 2 mins

In [ ]:
from geopy import distance

# Dividing by 2 to make half grid
# Dividing by 2 again to indicate that each aircraft in grid can move equal distance

# Considering longitudinal direction of 0.5
print(distance.distance((40.7, -73.8), (41.7, -73.8))/2/2)
# Considering latitudinal direction of 0.7
print(distance.distance((40.7, -73.8), (40.7, -72.4))/2/2)
# Considering increment in longitude and latitudes
print(distance.distance((40.7, -73.8), (41.7, -72.4))/2/2)

# 27.75 km / min = 1650+ km / hr > speed of aircraft. So half width of 2 * 0.5 lon, 2 * 0.7 lat looks like a good increment

Observations

- Unit change in x corresponds to approximately 0.1 change in longitude
- Unit change in y corresponds to approximately 0.1 change in latitude
- Unit change in z corresponds to approximately 500 ft change in altitude

In [ ]:
shifted = flight_data[["id", "ts", "x", "y", "z"]].set_index(['id']).groupby(level="id").shift(1)
flight_data['shifted_ts'] = shifted['ts'].reset_index(drop = True)
flight_data['shifted_x'] = shifted['x'].reset_index(drop = True)
flight_data['shifted_y'] = shifted['y'].reset_index(drop = True)
flight_data['shifted_z'] = shifted['z'].reset_index(drop = True)

In [ ]:
ts_1_shift_df = flight_data[flight_data['ts'] - flight_data['shifted_ts'] == 1]
(ts_1_shift_df['shifted_z'] - ts_1_shift_df['z']).describe(percentiles = [.01, .1, .25, .5, .75, .9, .99])

## Space at a given time

The space at a given time (t) is a 100 * 100 * 100 vector. Each cell of the vector represents the number of aircraft at the cell between (t - ts_range) and (t). This padding is done to avoid issues with missing data: therefore the number of unique aircraft-timestamp combinations in the total space will not be less than the number of rows of the raw data. If ts_range = 0, this should give the same number of unique aircraft-timestamp combinations as the number of rows in raw data (this is the desired outcome, not tested).

In [ ]:
def get_range_df_with_params(rng):
    return get_range_df(rng, flight_data, x_length, y_length, z_length, ts_range)

# Space for a timestamp remains the same. Only the action is continuous
if os.path.isfile('ts_full3d_space.bin'):
    space_X = pickle.load(open('ts_full3d_space.bin', 'rb'))
else:
    p = mp.Pool(processes = splits)
    # Arranging in reverse order to speed up computation
    rng = list(reversed(range(min(flight_data['ts']), max(flight_data['ts']) + 1)))
    split_rng = np.array_split(rng, splits)
    pool_results = p.map(get_range_df_with_params, split_rng)
    p.close()
    p.join()
    space_X = np.concatenate(pool_results, axis = 0)
    space_X = np.flip(space_X, axis = 0)
    pickle.dump(space_X, open("ts_full3d_space.bin", "wb"))


## Aircraft centering

At each time step, we center around a 'controlled' aircraft (including the padded aircraft for which the previous known location between (t - ts_range) and (t) is used). Space around aircraft is a (2 * half_x_length + 1, 2 * half_y_length + 1, 2 * half_z_length + 1) matrix with each cell representing the number of aircraft around the controlled aircraft at (x_cell - x_controlled, y_cell - y_controlled, z_cell - z_controlled) + (half_x_length, half_y_length, half_z_length). The controlled aircraft position is (half_x_length, half_y_length, half_z_length) for each aircraft-timestamp combination.

In [ ]:
# The following code is commented because outcome cannot be defined

# ac_ts_df = []
# for i in range(len(space_X)):
#     print(i)
#     ts_mat = space_X[i].toarray()
#     values = space_X[i].values
#     for aircraft in values:
#         # This operation can be reduced if we use numpy
#         # Sparse3dArray requires a check whether the key is within the boundary, which makes it slow
#         lower_x = aircraft[0] - half_x_length
#         upper_x = aircraft[0] + half_x_length + 1
#         lower_y = aircraft[1] - half_y_length
#         upper_y = aircraft[1] + half_y_length + 1
#         lower_z = aircraft[2] - half_z_length
#         upper_z = aircraft[2] + half_z_length + 1
#         aircraft_space = ts_mat[max(0, lower_x):min(x_length, upper_x), max(0, lower_y):min(y_length, upper_y), max(0, lower_z):min(z_length, upper_z)]
#         aircraft_space = pad_zeros(aircraft_space, lower_x, lower_y, lower_z, upper_x, upper_y, upper_z)
#         if aircraft_space.shape != tuple(state_dim):
#             print(aircraft_space.shape)
        
#         aircraft_space[half_x_length, half_y_length, half_z_length] = -1
#         ac_ts_df.append(np_3darray_to_Sparse3dArray(aircraft_space))



In [ ]:
uniq_id = flight_data['id'].unique()

In [ ]:
if os.path.isfile("Y_test_aircraft_center_space_multiclass_action.bin"):
    X_train = pickle.load(open("X_train_aircraft_center_space_multiclass_action.bin", "rb"))
    Y_train = pickle.load(open("Y_train_aircraft_center_space_multiclass_action.bin", "rb"))
    X_test = pickle.load(open("X_test_aircraft_center_space_multiclass_action.bin", "rb"))
    Y_test = pickle.load(open("Y_test_aircraft_center_space_multiclass_action.bin", "rb"))
else:
    XY = get_X_Y(uniq_id, space_X, half_x_length, half_y_length, half_z_length, x_length, y_length, z_length, discrete_action = True)
    X, Y = fix_XY(XY)
    X_train, Y_train, X_test, Y_test = train_test_split(X, Y)
    pickle.dump(X_train, open("X_train_aircraft_center_space_multiclass_action.bin", "wb"))
    pickle.dump(Y_train, open("Y_train_aircraft_center_space_multiclass_action.bin", "wb"))
    pickle.dump(X_test, open("X_test_aircraft_center_space_multiclass_action.bin", "wb"))
    pickle.dump(Y_test, open("Y_test_aircraft_center_space_multiclass_action.bin", "wb"))

train_ac_index = [len(X[0]) for X in X_train]
test_ac_index = [len(X[0]) for X in X_test]
X_train = unlist(unlist(X_train))
Y_train = bind(unlist(Y_train))
Y_train = Y_train.astype(np.int16)
X_test = unlist(unlist(X_test))
Y_test = bind(unlist(Y_test))
Y_test = Y_test.astype(np.int16)

# Model
## Initialization

In [ ]:
state_dim = [x_length, y_length, z_length]
cnn = CNN_multiclass(state_dim, action_size, learning_rate)

In [ ]:
# len(ac_ts_df)

### Train-test split

In [ ]:
# X_train = 

# Section on using the CNN class and data for training, prediction and evaluation

In [ ]:
saver = tf.train.Saver()
perc = 0
epochs = 1
batch_size = 128
num_batches = len(X_train)//batch_size

In [ ]:
completed = 0
epoch_train_losses = []
epoch_test_losses = []
ended = False
epoch = 0
sess = tf.Session()
# Initialize variables
sess.run(tf.global_variables_initializer())
while (epoch < epochs) and not(ended):
    completed = int(epoch * 100/epochs)
    if completed >= perc:
        logging.info(str(perc) + " % completed")
        perc = int(epoch * 100/epochs)

    batch_completed = 0
    mini_batch = 1
    mini_batch_train_losses = []
    mini_batch_test_losses = []
    while batch_completed < (len(X_train) - batch_size):
        train_X = X_train[batch_completed:(batch_completed + batch_size)]
        train_Y = Y_train[batch_completed:(batch_completed + batch_size)]
        loss = train(sess, cnn, train_X, train_Y)
        # Runs out of memory while evaluating on the complete test set. Only batch_size used for evaluating
        # This step should be randomized
        test_loss = get_loss(sess, cnn, X_test[0:batch_size], Y_test[0:batch_size])
        print('Epoch: {}/{}: '.format(epoch+1, epochs), 'mini-batch {}/{}: '.format(mini_batch, num_batches), "Train loss: {} ".format(loss), "Test loss: {} ".format(test_loss))
        batch_completed += batch_size
        mini_batch += 1
        mini_batch_train_losses.append(loss)
        mini_batch_test_losses.append(test_loss)
    epoch_train_losses.append(sum(mini_batch_train_losses)/len(mini_batch_train_losses))
    epoch_test_losses.append(sum(mini_batch_test_losses)/len(mini_batch_test_losses))
    # Early stopping check (callback should be used instead):
    if (epoch > 0) and (epoch_test_losses[epoch] > epoch_test_losses[epoch - 1]):
        saver.save(sess, "aircraft_center_multiclass")
        ended = True

    epoch += 1